# Working with Redis as a Vector Store

This notebook demonstrates the usage of the `RedisVectorStore` class from the langchain-redis package to enable efficient storage, retrieval, and similarity search of vector embeddings using Redis.

## Installation

In [ ]:
%pip install ipywidgets
%pip install langchain
%pip install langchain-redis
%pip install langchain-huggingface
%pip install sentence-transformers

## Importing Required Libraries

In [ ]:
import json
import redis

from redisvl.query.filter import Num
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_redis import RedisVectorStore

## Setting up Redis Connection

In [ ]:
REDIS_URL = "redis://localhost:6379"
redis_client = redis.from_url(REDIS_URL)
redis_client.ping()

## Preparing Sample Data

In [ ]:
with open('data/movies.json', 'r') as file:
    data = json.load(file)
    documents = [
        Document(
            page_content=movie["info"]["plot"],
            metadata={
                "title": movie["title"],
                "year": movie["year"],
                "rating": movie["info"]["rating"],
            })
        for movie in data["movies"]
    ]

Let's inspect the first document:

In [ ]:
documents[0]

## Creating Embeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="msmarco-distilbert-base-v4")

## Using Redis with LangChain
Now let's play with Redis as a vector store using LangChain.

### Creating a vector store instance and inserting data


In [ ]:
vector_store = RedisVectorStore.from_documents(
    documents,
    embeddings,
    redis_url=REDIS_URL,
    index_name="movies",
    metadata_schema=[
        {"name":"title", "type":"text"},
        {"name":"year", "type":"numeric"},
        {"name":"rating", "type":"numeric"},
    ],
    storage_type = "json"
)

### Performing a simple similarity search

In [ ]:
query = "He seeks revenge for his family's death"
results = vector_store.similarity_search(query, k=2)

for doc in results:
    print(doc)

### Similarity search with metadata filtering

In [ ]:
filter_condition = Num("year") == 2004

filtered_results = vector_store.similarity_search(
    query, k=2, filter=filter_condition
)

for doc in filtered_results:
    print(doc)

### Similarity search with score

In [ ]:
scored_results = vector_store.similarity_search_with_score(
    query, k=2, filter=filter_condition
)

for doc, score in scored_results:
    print(doc, score)

## Cleanup

In [ ]:
# Delete the underlying index and it's data
vector_store.index.delete(drop=True)